# DATA3404 Big Data Assignment Bootstrap

### Retrieve data from Google Drive and upload them into Databricks FileStore.
We perform these steps initially so that the data persists. If we download them locally into the cluster, the data will not persist after the cluster has been terminated.

In [0]:
dbfs_fileStore_prefix = "/FileStore/tables"
prefix = "ontimeperformance"
files = [
  { "name": f"{prefix}_flights_small.csv",   "file_id": "1LR5ULeE30oKo6DidtkHQNjCXJuZ-IIJO" },
  { "name": f"{prefix}_flights_medium.csv",  "file_id": "1VBn-BdOX7EGQrkMg7MkcoOEPDHYBNdL_" },
  { "name": f"{prefix}_flights_large.csv", "file_id": "1-KIBLcswJyzrbc4NsMrtVotyB1k2SdGX" },
  { "name": f"{prefix}_airlines.csv", "file_id": "1MIwg67fN43cK-frskA-2aYtxhw4rV9RY" },
  { "name": f"{prefix}_airports.csv", "file_id": "1cltP3m7Qgrp0Bei2LUlaYXwtsTz2wyCt" },
  { "name": f"{prefix}_aircrafts.csv", "file_id": "1JeDwF_zeJgE1Ebnnzkz2fRy1ZE2C7A6i" }
]

large_files =  { "name": f"{prefix}_flights_large.zip", "file_id": "1A7o60Qu62TherJRPyfMXAtCHIarS78oA" }

### Remove existing files

In [0]:
import os
for file in files:
  if os.path.exists("/tmp/{}".format(file["name"])):
    os.remove("/tmp/{}".format(file["name"]))
  dbutils.fs.rm("/FileStore/tables/{}".format(file["name"]))

### Download googledrivedownloader

In [0]:
%sh 
pip install googledrivedownloader

Collecting googledrivedownloader
 Downloading googledrivedownloader-0.4-py2.py3-none-any.whl (3.9 kB)
Installing collected packages: googledrivedownloader
Successfully installed googledrivedownloader-0.4
WARNING: You are using pip version 20.0.2; however, version 21.1.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python3.7 -m pip install --upgrade pip' command.

### Download small, medium and large (1GB) datasets to your DBFS

In [0]:
from google_drive_downloader import GoogleDriveDownloader as gdd

for file in files:
  gdd.download_file_from_google_drive(file_id=file["file_id"], dest_path="/tmp/{}".format(file["name"]))

Downloading 1LR5ULeE30oKo6DidtkHQNjCXJuZ-IIJO into /tmp/ontimeperformance_flights_small.csv... Done.
Downloading 1VBn-BdOX7EGQrkMg7MkcoOEPDHYBNdL_ into /tmp/ontimeperformance_flights_medium.csv... Done.
Downloading 1-KIBLcswJyzrbc4NsMrtVotyB1k2SdGX into /tmp/ontimeperformance_flights_large.csv... Done.
Downloading 1MIwg67fN43cK-frskA-2aYtxhw4rV9RY into /tmp/ontimeperformance_airlines.csv... Done.
Downloading 1cltP3m7Qgrp0Bei2LUlaYXwtsTz2wyCt into /tmp/ontimeperformance_airports.csv... Done.
Downloading 1JeDwF_zeJgE1Ebnnzkz2fRy1ZE2C7A6i into /tmp/ontimeperformance_aircrafts.csv... Done.

In [0]:
for file in files:
  dbutils.fs.mv("file:/tmp/{}".format(file["name"]), "/FileStore/tables/{}".format(file["name"]))


__STOP HERE__

The next step will take several minutes and the massive dataset loaded there is only needed for the scalability evaluation at the end of your assignment. For starting on your assignment task, you are fine with the first three datasets loaded so far.

### Download extra large files (8 GB), extract, and move them to DBFS

Only do the next step once you need this massive 8GB dataset for the actual performance evaluation. It will take a long time...

In [0]:
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id=large_files["file_id"], 
                                    dest_path="/tmp/{}".format(large_files["name"]),
                                    unzip=True)

In [0]:
dbutils.fs.mv("file:/tmp/8000mb_sample_flightdata.csv", f"{dbfs_fileStore_prefix}/{prefix}_flights_massive.csv")
